# Test script for SomaNews Clustering
By Datetime : 2016-08-29 ~ 2016-09-05

In [1]:
import numpy as np
import pandas as pd

## Load Data
Load Data from database

In [2]:
train = pd.read_pickle("../datastore/weekly.p")
train = train.drop(['newsNum'], axis=1)
train.head()

,category,content,provider,publishedAt,title
publishedAt,,,,,
2016-09-04 23:28:00,경제,모든 사물이 인터넷으로 연결된다는 사물인터넷(IoT·Internet of Thing...,chosun,2016-09-04 23:28:00,電氣 저장했다 꺼내 쓰는 ESS… 4년 뒤 150억달러 시장 선다
2016-09-04 23:24:00,nativeAD > 기업뉴스,지난달 말 산업통상자원부는 ESS 관련 산업 도약을 위해 민·관·학계가 함께하는 '...,chosun,2016-09-04 23:24:00,ESS 할인 요금제 기간 10년으로
2016-09-04 23:20:00,경제,정유업계에 긴장감이 돌고 있다. 올 상반기에는 세계 경기 불황 속에서도 사상 최고 ...,chosun,2016-09-04 23:20:00,중국發 공급과잉·低유가 먹구름… 잘나가던 정유업계 긴장
2016-09-04 22:54:00,경제,지난 3일(현지 시각) 오후 유럽 최대 가전박람회 'IFA 2016'이 열리고 있는...,chosun,2016-09-04 22:54:00,레고 로봇·스마트 캐디·게임 칫솔… 獨서 빛나는 작은 기업들
2016-09-04 22:52:00,nativeAD > LG전자,"""기술력에 대한 자신감이 없다면 우리 가전제품의 핵심 부품을 다른 경쟁 업체에 판매...",chosun,2016-09-04 22:52:00,"""경쟁社에 핵심부품 파는 건, 기술력에 자신감 있기 때문"""


## Preprocessing
0. Datetime (16-09-11 ~ 16-09-17)
1. Remove stopwords (regex, hanja)
2. POS Tagging with KoNLPy, Mecab

In [15]:
import datetime
from konlpy.tag import Mecab
import hanja
import cnouns
import re

In [4]:
mecab = Mecab()
print(mecab.pos(u'네, 안녕하세요 나는 정답'))

[('네', 'IC'), (',', 'SC'), ('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EP+EF'), ('나', 'NP'), ('는', 'JX'), ('정답', 'NNG')]


In [5]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', '', text)
    text = text.replace(u"카드뉴스", '').replace(u"조선일보", '')
    return text

### POS Tagging
일반명사, 고유명사, 외국어 (NNG, NNP, SL)

In [18]:
train['title'] = train['title'].apply(lambda text: text_cleaning(text))
title = [cnouns.tokenize(each[1]['title']) for each in train.iterrows()]


In [20]:
train['content'] = train['content'].apply(lambda text: text_cleaning(text))
content = [cnouns.tokenize(each[1]['content']) for each in train.iterrows()]

## Training
K-Means Algorithm

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from time import time

In [25]:
vectorizer = TfidfVectorizer(lowercase=False)
x_list = vectorizer.fit_transform(title)

In [26]:
vectorizer = TfidfVectorizer(lowercase=False)
x_list2 = vectorizer.fit_transform(content)

### Scoring

In [27]:
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.spatial.distance import cdist, pdist

### Best Silhoutte Score

In [28]:
best_score = 0.0
best_k = 0

In [31]:
for k in range(25, 35):
    km = KMeans(n_clusters=k, n_jobs=-1).fit(x_list)
    score = silhouette_score(x_list, km.labels_)
    if best_score < score:
        best_score = score
        best_k = k
    print("In Clusters =", k, ", Score is : %0.3f" % score)
print("In Clusters =", best_k, ", Best score is : %0.3f" % best_score)

In Clusters = 25 , Score is : 0.013
In Clusters = 26 , Score is : 0.015
In Clusters = 27 , Score is : 0.016
In Clusters = 28 , Score is : 0.014
In Clusters = 29 , Score is : 0.016
In Clusters = 30 , Score is : 0.014
In Clusters = 31 , Score is : 0.013
In Clusters = 32 , Score is : 0.016
In Clusters = 33 , Score is : 0.016
In Clusters = 34 , Score is : 0.015
In Clusters = 29 , Best score is : 0.016


### TODO: Using Elbow method

In [32]:
def elbow(df, n):
    kMeansVar = [KMeans(n_clusters=k).fit(df) for k in range(1, n)]
    centroids = [X.cluster_centers_ for X in kMeansVar]
    k_euclid = [cdist(df, cent) for cent in centroids]
    dist = [np.min(ke, axis=1) for ke in k_euclid]
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(df)**2)/df.shape[0]
    bss = tss - wcss
    plt.plot(bss)
    plt.show()

### K-Means Algorithm

In [33]:
from matplotlib import pyplot

In [34]:
t0 = time()
km = KMeans(n_clusters=best_k, n_jobs=-1).fit(x_list)
labels = km.labels_
centroids = km.cluster_centers_
print("Done in %0.3fs." % (time() - t0))

Done in 0.781s.


In [35]:
print(km.inertia_)

2926.761725945528


In [36]:
train['cluster'] = labels
train.head()

,category,content,provider,publishedAt,title,cluster
publishedAt,,,,,,
2016-09-04 23:28:00,경제,모든 사물이 인터넷으로 연결된다는 사물인터넷IoTInternet of Things에...,chosun,2016-09-04 23:28:00,전기 저장했다 꺼내 쓰는 ESS 4년 뒤 150억달러 시장 선다,4
2016-09-04 23:24:00,nativeAD > 기업뉴스,지난달 말 산업통상자원부는 ESS 관련 산업 도약을 위해 민관학계가 함께하는 ESS...,chosun,2016-09-04 23:24:00,ESS 할인 요금제 기간 10년으로,4
2016-09-04 23:20:00,경제,정유업계에 긴장감이 돌고 있다 올 상반기에는 세계 경기 불황 속에서도 사상 최고 영...,chosun,2016-09-04 23:20:00,중국발 공급과잉저유가 먹구름 잘나가던 정유업계 긴장,4
2016-09-04 22:54:00,경제,지난 3일현지 시각 오후 유럽 최대 가전박람회 IFA 2016이 열리고 있는 독일 ...,chosun,2016-09-04 22:54:00,레고 로봇스마트 캐디게임 칫솔 독서 빛나는 작은 기업들,14
2016-09-04 22:52:00,nativeAD > LG전자,기술력에 대한 자신감이 없다면 우리 가전제품의 핵심 부품을 다른 경쟁 업체에 판매하...,chosun,2016-09-04 22:52:00,경쟁사에 핵심부품 파는 건 기술력에 자신감 있기 때문,4


In [44]:
cluster = train.loc[train['cluster']==13]
cluster.title

publishedAt
2016-09-04 22:40:00                  원자재값 급등중규제 국내 배터리사 3중고
2016-09-04 22:40:00                  원자재값 급등중규제 국내 배터리사 3중고
2016-09-03 01:25:00         갤노트7 250만대 전량 교환 삼성 리콜 총비용 2조원대
2016-09-02 02:32:00                       삼성전자 갤럭시노트7 전량 리콜
2016-09-01 03:05:00          갤노트 7 폭발 주장 잇따라 삼성 국내 공급 일시 중단
2016-08-31 10:27:00                   삼성SDI 헝가리에 전기차 배터리 공장
2016-09-03 00:00:00           사설이재용 삼성의 위기관리 시험대 갤노트7 전량 리콜
2016-09-02 03:00:00                          배터리 논란 갤노트7 리콜
2016-09-01 03:00:00                      배터리 폭발논란 갤노트7 공급중단
2016-08-31 03:00:00             삼성SDI 4000억 들여 헝가리 배터리공장 설립
2016-08-29 03:00:00            갤노트7의 모든 것 보여드려요 3일까지 세빛섬 행사
2016-09-04 22:17:00                   갤노트7 배터리 점검 평소 2배로 늘어
2016-09-04 17:11:00         갤노트7 배터리 점검 평소 2배삼성 측 배터리 불량 없어
2016-09-03 14:27:00             제 배터리는 문제 없나요갤노트7 점검 고객 줄이어
2016-09-02 17:07:00              삼성전자 갤럭시노트7 250만대 신제품 교환환불
2016-09-02 17:06:00             삼성전자 갤노트7 판매 중단 전량 리콜 추후 공지
2016-09-02 15:19:00         

### Topic Modeling for contents
Latent Dirichlet Allocation (LDA)

In [45]:
from sklearn.decomposition import LatentDirichletAllocation

In [46]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [47]:
t0 = time()
lda = LatentDirichletAllocation(n_topics=1, max_iter=5, n_jobs=-1).fit(x_list2[5])
print("Done in %0.3fs." % (time() - t0))

Done in 0.791s.


In [48]:
feature_names = vectorizer.get_feature_names()
print_top_words(lda, feature_names, 5)

Topic #0:
감독이연출 상반기기준한국 한나라당수석 청와대의태도 과정예산



In [49]:
train['content'][5]

'윤부근 사진  삼성전자 소비자가전CE 부문 대표이사는 3일현지 시각 올해 내에 삼성 스마트TV에 탑재된 사물인터넷IoT 기능을 본격 강화해 진정한 스마트 허브를 구현하는 서비스를 내놓겠다고 말했다 윤 대표는 특히 앞으로 가전 업계는 누가 사물인터넷을 통해 패러다임 전환을 주도해 나가느냐가 사업의 생사를 가를 것이라며 사물인터넷 리더십 확보를 향후 핵심 전략으로 꼽았다\n\n윤 대표는 이날 유럽 최대 가전 전시회 IFA 2016이 열리는 독일 베를린의 삼성전자 전시관에서 기자와 만나 모바일과 TV냉장고 등 가전제품이 자연스럽게 연결되는 방향으로 사물인터넷 제품을 출시하겠다며 지금은 TV와 모바일 기기를 연결하려면 사람이 인위적으로 조작해야 하지만 앞으로는 TV 옆으로 가기만 해도 자동으로 연결돼 스마트폰을 TV 리모컨처럼 사용할 수 있을 것이라고 말했다\n\n삼성전자는 올해 처음으로 IoT 허브를 탑재한 스마트TV를 출시한 데 이어 앞으로 냉장고와 세탁기 등 다른 제품과의 연결성을 강화해 가정의 모든 기기를 연결하는 스마트홈을 본격적으로 구현하겠다는 전략이다 윤 대표는 올해 하반기에 가전 최대 격전지인 미국 시장에서 TV오디오스마트폰을 연결하는 사물인터넷 서비스를 시범 실시한 뒤 단계적으로 연결되는 가전제품과 서비스 국가를 확대할 계획이라고 말했다\n\n윤 대표는 사물인터넷이 23년 후에는 전자 산업의 패러다임을 바꿀 것으로 내다봤다 윤 대표는 현재는 기기를 연결해 제어하는 초기 단계지만 사물인터넷의 꽃은 각종 데이터를 분석한 뒤 소비자에게 다양한 서비스를 제공하는 것이라며 사물인터넷은 물류제조농업교통건강 등 모든 분야에 활용할 수 있기 때문에 앞으로 23년이 지나면 지금의 하드웨어 업체가 과연 살아있을까 하는 생각이 들 정도로 파괴력이 클 것이라고 말했다\n\n한편 윤 대표는 지난달 인수한 미국의 빌트인builtin붙박이 전문 고급 주방가전 업체인 데이코Dacor를 미국뿐만 아니라 유럽에서도 수퍼 프리미엄 브랜드로 내세워 프리미엄 가전과 기업 간 거래B2B 

## Result
1. Model Selection
2. Preprocessing
3. Feature Engineering
4. Parameter Tuning
5. Score Function